In [12]:
import socket
import threading
from typing import Tuple

class DeviceHandler:
    def __init__(self, ip: str, port: int):
        self.ip = ip
        self.port = port

    def read_data(self) -> str:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((self.ip, self.port))
            s.sendall(b'read')
            data = s.recv(1024)
        return data.decode()

    def write_data(self, data: str) -> None:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((self.ip, self.port))
            s.sendall(data.encode())

class DeviceManager:
    def __init__(self):
        self.devices = []

    def add_device(self, ip: str, port: int) -> None:
        device = DeviceHandler(ip, port)
        self.devices.append(device)

    def read_all_devices(self) -> None:
        threads = []
        for device in self.devices:
            thread = threading.Thread(target=self._read_device, args=(device,))
            threads.append(thread)
            thread.start()

        for thread in threads:
            thread.join()

    def write_all_devices(self, data: str) -> None:
        threads = []
        for device in self.devices:
            thread = threading.Thread(target=self._write_device, args=(device, data))
            threads.append(thread)
            thread.start()

        for thread in threads:
            thread.join()

    def _read_device(self, device: DeviceHandler) -> None:
        data = device.read_data()
        print(f"Read from {device.ip}:{device.port} - {data}")

    def _write_device(self, device: DeviceHandler, data: str) -> None:
        device.write_data(data)
        print(f"Wrote to {device.ip}:{device.port} - {data}")

In [13]:
def start_server(ip: str, port: int):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((ip, port))
        s.listen()
        print(f"Server started at {ip}:{port}")
        while True:
            conn, addr = s.accept()
            with conn:
                print(f"Connected by {addr}")
                data = conn.recv(1024)
                if data == b'read':
                    conn.sendall(b"sample data")
                else:
                    print(f"Received data: {data.decode()}")
                    conn.sendall(b"ack")

# Start server in separate threads for testing
server_thread_1 = threading.Thread(target=start_server, args=("127.0.0.1", 8080))
server_thread_1.start()

server_thread_2 = threading.Thread(target=start_server, args=("127.0.0.1", 8081))
server_thread_2.start()

Exception in thread Exception in thread Thread-12 (start_server):
Traceback (most recent call last):
  File "C:\Users\sahna\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
Thread-11 (start_server):
Traceback (most recent call last):
  File "C:\Users\sahna\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\sahna\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self.run()
  File "C:\Users\sahna\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\sahna\AppData\Local\Temp\ipykernel_7616\546661165.py", line 3, in start_server
    self._target(*self._args, **self._kwargs)
  File "C:\Users\sahna\AppData\Local\Temp\ipykernel_7616\546661165.py", line 3, in start_server
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is n

In [14]:
import unittest

class TestTCPCommunication(unittest.TestCase):
    def setUp(self):
        self.manager = DeviceManager()
        self.manager.add_device("127.0.0.1", 8080)
        self.manager.add_device("127.0.0.1", 8081)

    def test_read_data(self):
        self.manager.read_all_devices()

    def test_write_data(self):
        self.manager.write_all_devices("test data")

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.008s

OK


Connected by ('127.0.0.1', 58438)Connected by ('127.0.0.1', 58439)

Read from 127.0.0.1:8080 - sample data
Read from 127.0.0.1:8081 - sample data
Connected by ('127.0.0.1', 58440)
Received data: test data
Wrote to 127.0.0.1:8080 - test data
Connected by ('127.0.0.1', 58441)
Received data: test data
Wrote to 127.0.0.1:8081 - test data
